# Solution: The Forgotten Context

This notebook provides the complete solution to the debug drill.

---

In [ ]:
import re

In [ ]:
# Sample conversation that exceeds context window
conversation = [
    {"role": "user", "content": "Hi, I need help with order ORD-12345"},
    {"role": "assistant", "content": "I'd be happy to help with order ORD-12345. What's the issue?"},
    {"role": "user", "content": "The item arrived damaged. Can I get a replacement?"},
    {"role": "assistant", "content": "I'm sorry to hear that. Yes, you're eligible for a free replacement."},
    {"role": "user", "content": "Great! Can you also check my account for discounts?"},
    {"role": "assistant", "content": "Let me check your account for any available discounts."},
    {"role": "user", "content": "Also, I was charged twice last month."},
    {"role": "assistant", "content": "I'll investigate the billing issue. Can you confirm your email?"},
    {"role": "user", "content": "It's user@example.com. When will the replacement arrive?"},
]

MAX_CONTEXT_TOKENS = 100  # Small for demo

In [ ]:
def count_tokens(text):
    return int(len(text.split()) * 1.3)

In [ ]:
# ===== SOLUTION: Extract key facts =====

def extract_key_facts(messages):
    """Extract important facts that should never be forgotten."""
    facts = {}
    
    for msg in messages:
        content = msg['content']
        
        # Extract order numbers
        order_match = re.search(r'ORD-\d+', content)
        if order_match:
            facts['order_id'] = order_match.group()
        
        # Extract email
        email_match = re.search(r'[\w.-]+@[\w.-]+\.\w+', content)
        if email_match:
            facts['email'] = email_match.group()
        
        # Extract issues mentioned
        if 'damaged' in content.lower():
            facts['issue'] = 'damaged item'
        if 'replacement' in content.lower():
            facts['resolution'] = 'replacement requested'
    
    return facts

facts = extract_key_facts(conversation)
print("=== Extracted Key Facts ===")
for key, value in facts.items():
    print(f"  {key}: {value}")

In [ ]:
# ===== SOLUTION: Smart context with preserved facts =====

def smart_context(messages, facts, max_tokens=MAX_CONTEXT_TOKENS):
    """
    Build context that preserves key facts.
    """
    # Create system context with facts
    facts_summary = "Key facts from this conversation:\n"
    for key, value in facts.items():
        facts_summary += f"- {key}: {value}\n"
    
    system_msg = {"role": "system", "content": facts_summary}
    system_tokens = count_tokens(facts_summary)
    
    # Fill remaining space with recent messages
    remaining_tokens = max_tokens - system_tokens
    recent_messages = []
    
    for msg in reversed(messages):
        msg_tokens = count_tokens(msg['content'])
        if msg_tokens <= remaining_tokens:
            recent_messages.insert(0, msg)
            remaining_tokens -= msg_tokens
        else:
            break
    
    return [system_msg] + recent_messages

smart = smart_context(conversation, facts)

print("=== Smart Context (Facts Preserved) ===")
for msg in smart:
    print(f"{msg['role']}: {msg['content'][:60]}..." if len(msg['content']) > 60 else f"{msg['role']}: {msg['content']}")

In [ ]:
# Verify solution
context_text = ' '.join([m['content'] for m in smart])
assert 'ORD-12345' in context_text, "Order ID should be preserved"
print("\n✓ Order ID preserved in context!")
print("✓ Bot will not ask for order number again")

## Solution Summary

**Problem:** Bot forgets early context (order numbers, emails) when conversation is long

**Solution:**
1. **Extract key facts** using regex patterns (order IDs, emails, issues)
2. **Add facts to system prompt** so they're always in context
3. **Fill remaining space** with recent messages

**Result:** Critical information is never lost, even in long conversations